## 词嵌入（word2vec）

对于词典中的单词而言，one-hot编码方式无法衡量不同单词之间的相似度（任何两个不同词的one-hot向量的余弦相似度$\frac{\boldsymbol{x}^\top \boldsymbol{y}}{\Vert \boldsymbol{x} \Vert \cdot \Vert \boldsymbol{y} \Vert}$都为0）。

word2vec工具的提出正是为了解决上面这个问题。它将每个词表示成一个定长的向量，并使得这些向量能较好地表达不同词之间的相似和类比关系。word2vec工具包含了两个模型，即跳字模型（skip-gram）和连续词袋模型（continuous bag of words，CBOW）。
### 1 跳字模型
跳字模型假设基于某个词（定义为中心词）来生成它在文本序列周围的词（定义为背景词）。

例如，假设文本序列是“the”“man”“loves”“his”“son”。以“loves”作为中心词，设背景窗口大小为2，我们想要计算的是与它距离不超过2个词的背景词“the”“man”“his”“son”的条件概率。

![avatar](../resource/skip-gram.svg)

如何计算？

将每个词被表示成两个$d$维向量，假设这个词在词典中索引为$i$，当它为中心词时向量表示为$\vec{v}_i \in \mathbb{R}^d$，为背景词时向量表示为$\vec{u}_i \in \mathbb{R}^d$。设中心词$w_c$在词典中索引为$c$，背景词$w_o$在词典中索引为$o$，给定中心词生成背景词的条件概率可以通过对向量内积做softmax运算而得到：

$$
\mathbb{P} (w_o \mid w_c) = \frac{\exp (\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{\sum_{i \in \mathcal{V}} \exp (\boldsymbol{u}_i^\top \boldsymbol{v}_c)},
$$

其中$\mathcal{V}$为词典中所有单词的索引。

给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$。假设给定中心词的情况下背景词的生成相互独立，当背景窗口大小为$m$的时候，跳字模型的似然函数，即给定**任意**中心词生成**所有**背景词的概率，为

$$
\prod_{t=1}^T \prod_{-m \leq j \leq m, j \neq 0} \mathbb{P} (w^{(t+j)} \mid w^{(t)}).
$$

自动忽略大于$T$和小于1的时间步。

#### 训练跳字模型

跳字模型的参数是每个词所对应的中心词向量和背景词向量。通过最大化似然来更新参数。
最大化上文中的似然函数等价于最小化以下损失函数：

$$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, \mathbb{P} (w^{(t+j)} \mid w^{(t)}).$$


如果使用随机梯度下降，那么在每一次迭代中随机采样一个较短的子序列来计算有关该子序列的损失，然后计算梯度来更新模型参数。对于所有属于以$w_c$为中心词的窗口内的背景词$w_o$,


$$\log \mathbb{P} (w_o \mid w_c) =
\boldsymbol{u}_o^\top \boldsymbol{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)\right).$$

所以得到

$$
\begin{aligned}
\frac{\partial \text{log}\, \mathbb{P} (w_o \mid w_c)}{\partial \boldsymbol{v}_c} 
&= \boldsymbol{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\boldsymbol{u}_j^\top \boldsymbol{v}_c)\boldsymbol{u}_j}{\sum_{i \in \mathcal{V}} \exp(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\\
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\boldsymbol{u}_j^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\right) \boldsymbol{u}_j\\ 
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} \mathbb{P} (w_j \mid w_c) \boldsymbol{u}_j.
\end{aligned}
$$

它的计算需要词典中所有词以$w_c$为中心词的条件概率。
所以
$$
\boldsymbol{v}_c \leftarrow \boldsymbol{v}_c - \eta \Big( \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} \mathbb{P} (w_j \mid w_c) \boldsymbol{u}_j \Big).
$$

对于所有属于以$w_c$为中心词的窗口内的背景词$w_o$的向量$\boldsymbol{u}_o$，

$$
\frac{\partial \text{log}\, \mathbb{P} (w_o \mid w_c)}{\partial \boldsymbol{u}_o}
= \boldsymbol{v}_c - \mathbb{P} (w_o \mid w_c) \boldsymbol{v}_c.
$$

训练结束后，对于词典中的任一索引为$i$的词，我们均得到该词作为中心词和背景词的两组词向量$\boldsymbol{v}_i$和$\boldsymbol{u}_i$。**一般使用跳字模型的中心词向量作为词的表征向量**。

### 2 连续词袋模型

连续词袋模型与跳字模型类似。与跳字模型最大的不同在于，连续词袋模型假设基于某中心词在文本序列前后的背景词来生成该中心词。在同样的文本序列“the”“man”“loves”“his”“son”里，以“loves”作为中心词，且背景窗口大小为2时，连续词袋模型关心的是，给定背景词“the”“man”“his”“son”生成中心词“loves”的条件概率。

![avatar](../resource/cbow.svg)

因为连续词袋模型的背景词有多个，我们将这些背景词向量取平均，然后使用和跳字模型一样的方法来计算条件概率。设$\boldsymbol{v_i}\in\mathbb{R}^d$和$\boldsymbol{u_i}\in\mathbb{R}^d$分别表示词典中索引为$i$的词作为背景词和中心词的向量（注意符号的含义与跳字模型中的相反）。设中心词$w_c$在词典中索引为$c$，背景词$w_{o_1}, \ldots, w_{o_{2m}}$在词典中索引为$o_1, \ldots, o_{2m}$，那么给定背景词生成中心词的条件概率

$$P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\boldsymbol{u}_c^\top (\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\left(\frac{1}{2m}\boldsymbol{u}_i^\top (\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}}) \right)}.$$

为了让符号更加简单，我们记$\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$，且$\bar{\boldsymbol{v}}_o = \left(\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}} \right)/(2m)$，那么上式可以简写成

$$P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\boldsymbol{u}_c^\top \bar{\boldsymbol{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o\right)}.$$

给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$，背景窗口大小为$m$。连续词袋模型的似然函数是由背景词生成**任意**中心词的概率

$$ \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots,  w^{(t-1)},  w^{(t+1)}, \ldots,  w^{(t+m)}).$$

#### 训练连续词袋模型

连续词袋模型的最大似然估计等价于最小化损失函数

$$  -\sum_{t=1}^T  \text{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots,  w^{(t-1)},  w^{(t+1)}, \ldots,  w^{(t+m)}).$$

注意到

$$\log\,P(w_c \mid \mathcal{W}_o) = \boldsymbol{u}_c^\top \bar{\boldsymbol{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o\right)\right).$$

所以对于$\boldsymbol{v}_{o_i}$（$i = 1, \ldots, 2m$），

$$\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \boldsymbol{v}_{o_i}} = \frac{1}{2m} \left(\boldsymbol{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\boldsymbol{u}_j^\top \bar{\boldsymbol{v}}_o)\boldsymbol{u}_j}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o)} \right) = \frac{1}{2m}\left(\boldsymbol{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid\mathcal{W}_o) \boldsymbol{u}_j \right).$$

同理

$$
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \boldsymbol{u}_c} = \bar{\boldsymbol{v}}_o - \mathbb{P} (w_c \mid \mathcal{W}) \bar{\boldsymbol{v}}_o.
$$

同跳字模型不一样的一点在于，**一般使用连续词袋模型的背景词向量作为词的表征向量**。